<a href="https://colab.research.google.com/github/ritikaa05-svg/ai-solve/blob/main/8puzzle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import heapq

# Define the goal state for the 8 puzzle
goal_state = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 0]
]

# Directions for moving the empty space
DIRECTIONS = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # up, down, left, right

class PuzzleState:
    def __init__(self, board, parent=None, g=0):
        self.board = board  # current state of the puzzle
        self.parent = parent  # parent state
        self.g = g  # cost to reach this state (number of moves)
        self.h = self.calculate_heuristic()  # heuristic value (Manhattan distance)
        self.f = self.g + self.h  # total cost function f(n) = g(n) + h(n)

    def calculate_heuristic(self):
        """Calculate the Manhattan distance heuristic."""
        h = 0
        for i in range(3):
            for j in range(3):
                if self.board[i][j] != 0:
                    goal_i, goal_j = divmod(self.board[i][j] - 1, 3)
                    h += abs(goal_i - i) + abs(goal_j - j)
        return h

    def is_goal(self):
        """Check if the current state is the goal state."""
        return self.board == goal_state

    def get_empty_position(self):
        """Find the position of the empty space (0)."""
        for i in range(3):
            for j in range(3):
                if self.board[i][j] == 0:
                    return i, j
        return None

    def generate_children(self):
        """Generate all possible valid moves from the current state."""
        children = []
        x, y = self.get_empty_position()

        for dx, dy in DIRECTIONS:
            new_x, new_y = x + dx, y + dy

            # Check if the new position is within the bounds of the board
            if 0 <= new_x < 3 and 0 <= new_y < 3:
                new_board = [row[:] for row in self.board]  # Copy the current board
                # Swap the empty space with the tile in the new position
                new_board[x][y], new_board[new_x][new_y] = new_board[new_x][new_y], new_board[x][y]
                child_state = PuzzleState(new_board, self, self.g + 1)
                children.append(child_state)

        return children

    def __lt__(self, other):
        """Compare PuzzleState objects to prioritize in the heap."""
        return self.f < other.f


def a_star(start_state):
    """Solve the 8 puzzle using the A* algorithm."""
    open_list = []
    closed_list = set()

    # Push the start state onto the priority queue
    heapq.heappush(open_list, start_state)

    while open_list:
        current_state = heapq.heappop(open_list)

        # Check if the current state is the goal state
        if current_state.is_goal():
            # Reconstruct the path
            path = []
            while current_state:
                path.append(current_state.board)
                current_state = current_state.parent
            return path[::-1]  # Reverse the path to get the solution from start to goal

        closed_list.add(tuple(map(tuple, current_state.board)))  # Add the state to the closed list

        # Generate all valid children and add to open list
        for child in current_state.generate_children():
            if tuple(map(tuple, child.board)) not in closed_list:
                heapq.heappush(open_list, child)

    return None  # No solution found


# Define the initial state
initial_state = PuzzleState(
    [
        [2, 8, 3],
        [1, 6, 4],
        [7, 0, 5]
    ]
)

# Solve the puzzle using A* algorithm
solution_path = a_star(initial_state)

if solution_path:
    print(f"Solution found in {len(solution_path) - 1} moves:")
    for step in solution_path:
        for row in step:
            print(row)
        print()  # Print an empty line between steps
else:
    print("No solution found.")


No solution found.
